In [1]:
import collections
import pickle

import matplotlib
import numpy as np
import seaborn as sns
import sklearn

from matplotlib import pyplot as plt
from matplotlib import cm
from sklearn import manifold

sns.set_style("darkgrid")

In [2]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view, head_view
from transformers import BertModel, BertTokenizer
import pandas as pd
import torch
import sys

/home/aneta/Desktop/Code/enviroments_dir/tableml/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sys.path.append('../AIDAMA/tableml')

In [4]:
from taml.turl.model import HybridTableCT
from taml.turl.model import TableConfig
from taml.turl.tokenizer import BertTokenizer
from taml.turl.dataset import process_single_bf, process_single_CT
from taml.turl.dataloader import finetune_collate_fn_CT
from taml.preproc.utils import read_type_vocab

In [5]:
config_class = TableConfig
config = config_class.from_pretrained('../AIDAMA/tableml/pretrained/table-base-config_v2.json')
config.output_attentions = True
config.mode = 3
config.class_num = 1
config.label_conditioned = False
config.soft_labels = False

In [6]:
config_class, model_class = TableConfig, HybridTableCT
model = model_class(config, is_simple=True)
state_dict = torch.load('../AIDAMA/tableml/pretrained/pytorch_model.bin', map_location=torch.device('cpu'))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['cls.weight', 'cls.bias'], unexpected_keys=['cls.tok_predictions.bias', 'cls.tok_predictions.transform.dense.weight', 'cls.tok_predictions.transform.dense.bias', 'cls.tok_predictions.transform.LayerNorm.weight', 'cls.tok_predictions.transform.LayerNorm.bias', 'cls.tok_predictions.decoder.weight', 'cls.ent_predictions.transform.dense.weight', 'cls.ent_predictions.transform.dense.bias', 'cls.ent_predictions.transform.LayerNorm.weight', 'cls.ent_predictions.transform.LayerNorm.bias', 'cls.ent_predictions.bias.weight'])

In [7]:
config.max_input_tok = 500
config.max_title_length = 50
config.max_header_length = 50
config.max_cell_length = 50
config.max_column = 1
config.type_vocab = {}
config.tokenizer = tokenizer
config.label_df = pd.DataFrame([{'sheet': None, 'column': None}])

In [9]:
df = pd.read_csv('dummy4attention.csv')
df = df[0:1] # take just the first row

In [10]:
df

,Name,Auto Racing Team,Formula1 Race,Year
0,Michael Schumacher,Scuderia Ferrari,2005 United States Grand Prix,2005


In [11]:
sample = process_single_bf(('table id', test_df2), config)
table_id, input_tok, input_tok_type, input_tok_pos, input_tok_mask, input_ent_text, input_ent_text_length, input_ent, input_ent_type, input_ent_mask, column_entity_mask, column_header_mask, labels_mask, labels = finetune_collate_fn_CT(tokenizer, is_train=False)([sample])
print(input_tok.shape)

torch.Size([1, 8])


In [12]:
header_tokens = tokenizer.convert_ids_to_tokens([2171, 8285, 3868, 2136, 5675, 2487, 2679, 2095])  

In [13]:
row_tokens = ['[PAD]']
for rowIndex, row in test_df2.iterrows(): #iterate over rows
    for columnIndex, value in row.items():
        row_tokens.append(str(value).lower())

In [14]:
header_tokens.extend(row_tokens)

In [15]:
with torch.no_grad():
    model.eval()    
    tok_out, ent_out, ent_cand_embs = model.table(input_tok,
            input_tok_type,
            input_tok_pos,
            input_tok_mask,
            input_ent_text,
            input_ent_text_length,
            None,
            input_ent,
            input_ent_type,
            input_ent_mask,
            None)

In [16]:
num_all = tok_out[1][0].size(3)

In [17]:
num_header = tok_out[1][0].size(2)

In [18]:
attens = tok_out[1][0].shape

In [19]:
pad_tensor = torch.zeros(1,12,5,13)
pad_tensor_ent = torch.zeros(1,12,8,13)

In [20]:
len(tok_out[1])

4

In [21]:
token_probs = [torch.cat((tok_out[1][i], pad_tensor), 2) for i in range(len(tok_out[1]))]

In [22]:
ent_probs = [torch.cat((pad_tensor_ent, ent_out[1][i]),2)for i in range(len(ent_out[1]))]

In [23]:
model_view_token_html = model_view(token_probs, header_tokens, display_mode = "dark" )

<IPython.core.display.Javascript object>

In [25]:
model_view_token_html = model_view(ent_probs, header_tokens, display_mode = "dark" )

<IPython.core.display.Javascript object>

In [24]:
html_head_view_tok = head_view(ent_probs, header_tokens)

<IPython.core.display.Javascript object>